In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
from datasets import load_dataset
from google.colab import drive
drive.mount('/content/drive')


# Load dataset in CoNLL format
dataset = load_dataset("text", data_files="/content/drive/MyDrive/labeled_data.conll")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
!pip install huggingface_hub


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification

# Load your dataset
# Replace "your_dataset_name" with your actual dataset name
# dataset = load_dataset("your_dataset_name")

# Define num_classes based on your dataset labels
# For example, if you have two classes (e.g., 0 and 1):
num_classes = 2  # Update this based on your dataset

# Load the model and tokenizer
model_name = "xlm-roberta-base"  # Or another suitable XLM-RoBERTa variant
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenization function
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["text"],
        truncation=True,
        padding='max_length',
        max_length=512,
        return_offsets_mapping=True,
    )

    # Example: Assuming the labels are in the 'label' field of your dataset
    labels = examples.get("label", [0] * len(tokenized_inputs['input_ids']))  # Adjust based on your dataset
    tokenized_inputs["labels"] = labels

    return tokenized_inputs

# Tokenizing the dataset
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# Create train-test split
train_test_split = tokenized_datasets["train"].train_test_split(test_size=0.1)  # 10% for validation
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

# Set up the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

# Start training
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/2724 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.000024
2,0.011900,0.000013
3,0.011900,0.000010


TrainOutput(global_step=921, training_loss=0.006477796196549257, metrics={'train_runtime': 836.0413, 'train_samples_per_second': 8.795, 'train_steps_per_second': 1.102, 'total_flos': 1934655590062080.0, 'train_loss': 0.006477796196549257, 'epoch': 3.0})

In [ ]:
# Step 9: Evaluate the fine-tuned model
evaluation_results = trainer.evaluate()

# Print evaluation results
print("Evaluation results:", evaluation_results)

# Step 10: Save the model for future use
trainer.save_model("./ner_model")  # Save the fine-tuned model
tokenizer.save_pretrained("./ner_model")  # Save the tokenizer


Evaluation results: {'eval_loss': 1.005457579594804e-05, 'eval_runtime': 8.9974, 'eval_samples_per_second': 30.342, 'eval_steps_per_second': 3.89, 'epoch': 3.0}


('./ner_model/tokenizer_config.json',
 './ner_model/special_tokens_map.json',
 './ner_model/sentencepiece.bpe.model',
 './ner_model/added_tokens.json',
 './ner_model/tokenizer.json')